<a href="https://colab.research.google.com/github/bradygm/PID-controller-simulator/blob/main/PID_controller_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RoboLaunch Workshop Series: Intuitive PID Control

Link to recording: will be updated after live workshop session 
<!-- ![](https://drive.google.com/uc?export=view&id=1jj4E63su7JoKRsCWAJ73ZGNPCLApBT6a) -->

### Quick Google Colab Intro

A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

Below is a **code cell** that prints "Hello World!". Click in the cell to select it and execute the contents in one of the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); 

In [ ]:
# This is a comment
print('Hello World!')

Hello World!


Did you see the code cell print "Hello World!"? Great!

# Example 1: Single link robot arm

Just a stick with one actuated joint. They could have a slider that affects the commanded joint angle and then they see how the robot arm rotates to that angle. 

Physics would be very simple, as we could ignore gravity if arm is in x,y plane. They could even modify the mass of the arm to see how it affects the controller, but might be beyond the scope of this workshop. But we could have a section at the end that gives people ideas of things they could do if they want to explore it more and try other things. 

We could also hide away some of the code maybe, or maybe even remove classes?


#### Import and helpers statements (explain)

##### Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import math
from IPython.display import HTML, Javascript
from scipy import interpolate
from ipywidgets import interact, interactive, fixed, interact_manual, Video
import ipywidgets as widgets
import time

##### CustomInput Class

In [24]:
class CustomInput:

    def __init__(self, min_val, max_val):
        self.a = widgets.FloatSlider(description='Input',min=min_val, max=max_val)
        self.values = []
        self.xhist = []
        self.yhist = []

        self.start_time = -1.0
        self.first_time = True

    def f(self, a):
        if self.first_time:
            self.first_time = False
            plt.plot(self.xhist, self.yhist)
            plt.xlabel("Seconds")
            plt.ylabel("Input")
            return

        if self.start_time == -1.0:
            self.start_time = time.time()

        self.xhist.append(time.time()-self.start_time)
        self.yhist.append(a)
        self.values.append([a, time.time()-self.start_time])
        plt.plot(self.xhist, self.yhist, drawstyle="steps-pre")
        plt.xlabel("Seconds")
        plt.ylabel("Input")

    
    def record(self):
        self.values = []
        self.xhist = []
        self.yhist = []

        self.start_time = -1.0
        self.first_time = True

        out = widgets.interactive_output(self.f, {'a': self.a})

        # vid_out = widgets.Output(layout={'border': '1px solid black'})
        # with vid_out:
        #     display(Video.from_file("animation.mp4", play=True))



        # widgets.HBox([widgets.VBox([a]), out, vid_out])
        return widgets.HBox([widgets.VBox([self.a]), out])




In [69]:
#############################################
# Display the results
def display_results():
    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(-1.2, 1.2), ylim=(-1.2, 1.2))
    ax.set_aspect('equal')

    line, = ax.plot([], [], 'o-', lw=6)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


    def animate_arm(i):
        thisx = [0, np.sin(ang_hist[i])]
        thisy = [0, np.cos(ang_hist[i])]

        line.set_data(thisx, thisy)
        time_text.set_text(time_template % (i*myRobotArm.timestep))
        return line, time_text

    ani_arm_vid = animation.FuncAnimation(
        fig, animate_arm, len(ang_hist), interval=myRobotArm.timestep*1000, blit=True)
    plt.close()


    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
    plt.title("Arm Angle and Goal Angle")
    plt.xlabel("Seconds")
    plt.ylabel("Angle (Radians)")
    ax.grid()

    line2, = ax.plot([], [], '--', lw=2)
    line2.set_label('Arm Angle')
    line3, = ax.plot(time_list[0], ang_hist[0], 'o', lw=2)
    line4, = ax.plot(time_list[0:list_len-1], goal_ang_interp[0:list_len-1], '--', lw=2)
    line4.set_label('Arm Goal Angle')
    time_template2 = 'time = %.1fs'
    time_text2 = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    # ax.legend()


    def animate_arm2(i):
        line2.set_data(time_list[0:i], ang_hist[0:i])
        line3.set_data(time_list[i], ang_hist[i])
        time_text2.set_text(time_template2 % (i*myRobotArm.timestep))

        return line2, time_text2

    ani_arm_vid2 = animation.FuncAnimation(
        fig, animate_arm2, len(ang_hist), interval=myRobotArm.timestep*1000, blit=True)
    plt.close()

    #################################
    # display each animation next to each other
    window1 = widgets.Output(layout={'border': '1px solid black'})
    window2 = widgets.Output(layout={'border': '1px solid black'})
    start = time.time()
    with window1:
        if animate_sim:
            display(HTML(ani_arm_vid.to_html5_video()))
    with window2:
        if animate_plots:
            display(HTML(ani_arm_vid2.to_html5_video()))
        else:
            fig = plt.figure(figsize=(6, 4))
            ax = fig.add_subplot(autoscale_on=False, xlim=(0, end_time), ylim=(-np.pi, np.pi))
            ax.grid()
            plt.title("Arm Angle and Goal Angle")
            plt.xlabel("Seconds")
            plt.ylabel("Angle (Radians)")
            line2, = ax.plot(time_list[0:list_len-1], goal_ang_interp[0:list_len-1], '--', lw=2)
            line2.set_label('Arm Goal Angle')
            line3, = ax.plot(time_list[0:list_len-1], ang_hist[0:list_len-1], '--', lw=2)
            line3.set_label('Arm Angle')
            ax.legend()
            plt.show()
        
    print(time.time()-start)



    # widgets.HBox([widgets.VBox([a]), out, vid_out])
    return widgets.HBox([window1, window2])
    # FFwriter = animation.FFMpegWriter()
    # ani.save('animation.mp4', writer = FFwriter)

def interp_values(values, end_time, dt):
    values_t = np.array(values)
    f = interpolate.interp1d(values_t[:,1], values_t[:,0], kind='previous', fill_value="extrapolate")
    time_list = np.arange(0, end_time, dt)
    return f(time_list)

#### Create your own input for the arm

In [25]:
custom_torques = CustomInput(-1,1)
custom_torques.record()

In [60]:
custom_goal_angle = CustomInput(-3.14,3.14)
custom_goal_angle.record()

#### Controller


In [14]:
class PID_Controller:

    def __init__(self, k_p=1, k_i=0, k_d=0):
        self.control_limit = 0 #not currently used
        self.error_sum = 0

        self.k_p = k_p
        self.k_i = k_i
        self.k_d = k_d

    def p_control(self, desired_position, current_position):
        error = desired_position-current_position
        return self.k_p*error

    def pd_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        return self.k_p*error - self.k_d*current_velocity

    def pid_control(self, desired_position, current_position, current_velocity):
        error = desired_position-current_position
        self.error_sum += error
        return self.k_p*error + self.k_i*self.error_sum - self.k_d*current_velocity

In [27]:
# @markdown # Robot Arm Parameters { display-mode: "form" }
# @markdown You can adjust parameters using the slides and form below.
# @markdown This is pretty fun to interact with. Remember to run this 
# @markdown cell after you pick new parameters.

# @markdown ---
# @markdown ### Physical Parameters:

import numpy as np

class RobotArm:

    def __init__(self, start_angle, timestep):
        self.mass = 1.1 #@param{type:"slider", min:0.1, max:3, step:0.1}
        self.length = 1 #@param
        
        self.state = np.array([start_angle, 0.0])
        # self.ang_position = start_angle
        # self.ang_velocity = 0

        self.damping = 0.1 #@param
        self.gravity_on = False #@param{type:"boolean"}
        self.gravity = 9.8

        # @markdown ---
        # @markdown ### Simulation Parameters:
        self.timestep = timestep

    def state_derivative(self, state, torque):
        ang_position = state[0]
        ang_velocity = state[1]
        if self.gravity_on:
            accelaration = (3/self.mass/self.length**2)*(torque - self.damping*ang_velocity + self.mass*self.gravity*self.length/2*np.sin(ang_position))
        else:
            accelaration = 3*(torque-ang_velocity*self.damping)/(self.mass*self.length**2)

        return np.array([ang_velocity, accelaration])
        
        
    def step_forward(self, torque): 
        # Using Runge-Kutta algorithm
        k1 = self.state_derivative(self.state, torque)
        k2 = self.state_derivative(self.state + self.timestep/2*k1, torque)
        k3 = self.state_derivative(self.state + self.timestep/2*k2, torque)
        k4 = self.state_derivative(self.state + self.timestep*k3, torque)
        self.state = self.state + self.timestep/6 * (k1 + 2*k2 + 2*k3 + k4)

        # Euler method not stable
        # self.ang_position = self.ang_position + self.timestep*self.ang_velocity
        # self.ang_velocity = self.ang_velocity + self.timestep*accelaration

    def angle_wrap(self, ang):
        return (ang + np.pi) % (2 * np.pi) - np.pi

    def get_ang_position(self):
        return self.state[0]
    
    def get_ang_velocity(self):
        return self.state[1]

    


In [70]:



end_time = 10.0 #@param{type:"slider", min:5.0, max:30, step:1}
start_angle = 0.1 #@param{type:"slider", min:-3.14, max:3.14, step:0.1}
k_p = 6.71 #@param{type:"slider", min:0.0, max:20.0, step:0.01}
k_i = 0.0 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
k_d = 1.94 #@param{type:"slider", min:0.0, max:5.0, step:0.01}
dt = 0.05 #@param
animate_plots = True #@param{type:"boolean"}
animate_sim = True #@param{type:"boolean"}


myRobotArm = RobotArm(start_angle, dt)
controller = PID_Controller(k_p, k_i, k_d)

# time_list = np.arange(0, end_time, dt)
# list_len = len(time_list)
torques_interp = interp_values(custom_torques.values, end_time, dt)

goal_ang_interp = interp_values(custom_goal_angle.values, end_time, dt)


ang_hist = []

for i in range(len(torques_interp)):
    # myRobotArm.step_forward(torques_interp[i])
    # myRobotArm.step_forward(myRobotArm.p_control(.5))
    myRobotArm.step_forward(controller.pd_control(goal_ang_interp[i], myRobotArm.get_ang_position(), myRobotArm.get_ang_velocity()))
    # cur_torque = controller.pid_control(0, myRobotArm.get_ang_position(), myRobotArm.get_ang_velocity())
    # myRobotArm.step_forward(cur_torque)
    # myRobotArm.step_forward(0)
    ang_hist.append(myRobotArm.get_ang_position())

display_results()

18.52904224395752


Try turning gravity on or off. Start at angle and see what it does. 

<hr>

Then they mess around with the slider being the torque. We can then say how rather than a slider with us puting in the torque, we use a PID controller.

Allow for changing the trajectory. Also a slider? How easily change? Swap out or comment out lines?

# Example 2: Line following robot

Could start with or maybe only have a simple straight line. Car going from left to right. They modify the starting position laterally (starting crosstrack error). Maybe move onto a more interesting track or we have a set of tracks. Would be really cool if they could draw their own paths. 

Ideas
* Could have live plot showing cross track error (can be used for explaining the D or I of PID but also just fun to see)
* Have a benchmark set for them to get a score and maybe have leaderboard optional
* Fun parameters we could expose with sliders or text entry or could just be clearly marked in the code
  * Vehicle speed
  * Max turn rate/angle
  * Look ahead distance when calculating cross track error?

# Example 3? 
Probably two is fine, and maybe even just stick to one. But having the very simple arm example in the beginning might be really nice.

# Notes and thoughts

* How allow people new to coding to even participate?
  * Option to reveal the answers or get them somehow and copy and paste
* Make clear what prerequisits are needed
  * Internet Browser
* Could have a quick intro video they watch before the workshop that gives them an overview of google colab or other background info. They then come prepared. Could have a simple notebook they go through
* How make the quality of our presentation really good? 
  * Zoom clear background thing?
  * OBS?
  * Look at other examples?
* How are we going to run the interactive portion, especially with things being hybrid?
  * Will it be open or do they sort of follow along?
  * How check in?
  * Open and then run through together at end?
  * Breakout rooms?
    * How would that work with the recorded video?